In [1]:
import pandas as pd

In [94]:
now = "vila"
df = pd.read_csv("besede/stavki-{}.csv".format(now))
df

,Unnamed: 0,class,word,sentence1,sentence2
0,0,NaN,vila,....nekaj avtobobilom luks.razreda in vil .,"poslovnež, ki je do potankosti podrobno poskr..."
1,1,NaN,vila,... Mira sem obiskala deseti dan bivanja v v...,Samostojne vile
2,2,NaN,vila,"Tu živi,« je odvrnil Lindgren in pokazal prot...",Silvio Berlusconi. Objavil je fotografije z z...
3,3,NaN,vila,"procente in rabate, potem pa tridesetletni fr...","takoj ko je spregovoril začel s tem,da bi vi..."
4,4,NaN,vila,"Mikec in Damjanom Snojem, za njimi pa se je ...","na raziskovanje velikega kompleksa, ki se je ..."
...,...,...,...,...,...
495,495,NaN,vila,"minut vožnje od Opatije, je na ekskluzivni lo...",". Dolge kolone avtomobilov, ki so se konec te..."
496,496,NaN,vila,2. Je pravljično lepa vila v resnici Darkov...,"IZ KATEREGA JE MOLČE SNEŽILO, SE JE RAZPENJAL..."
497,497,NaN,vila,", za najem stavbe NPU, Hypo banko, gradnjo v...",dokaj na hitro in brez možnosti podaljšanja p...
498,498,NaN,vila,Vila pod hollywoodskim znakom,gospa Delopst pa da je prišla pred nje tudi z...


In [37]:
import simplemma
from nltk import word_tokenize
import string

langdata = simplemma.load_data('sl')

stopwords = []

with open("SloStopWords.txt", "r", encoding="utf-8") as st:
    lis = []
    for l in st.readlines():
        lis.append(l.strip())        
    stopwords = lis
print(stopwords[0])


def remove_punc(text):
    return "".join([char for char in text if char not in string.punctuation])

def remove_stops(text: list):
    for ind,i in enumerate(text):
        if i in stopwords:
            text.pop(ind)
    return text

def remove_nums(text):
    return ''.join([i for i in text if not i.isdigit()])

# df['sentence1_p'] = df['sentence1'].apply(lambda x: simplemma.lemmatize(x.lower(), langdata))

ali


In [95]:
df['sentence1_p'] = df['sentence1'].apply(lambda x: x.lower())
df['sentence1_p'] = df['sentence1_p'].apply(lambda x: " ".join(remove_nums(remove_punc(x)).split()))
df['sentence1_p'] = df['sentence1_p'].apply(lambda x: x.split(" "))
df['sentence1_p'] = df['sentence1_p'].apply(lambda x: remove_stops(x))
df['sentence1_p'] = df['sentence1_p'].apply(lambda x: [simplemma.lemmatize(t, langdata) for t in x])

df['sentence2_p'] = df['sentence2'].apply(lambda x: x.lower())
df['sentence2_p'] = df['sentence2_p'].apply(lambda x: " ".join(remove_nums(remove_punc(x)).split()))
df['sentence2_p'] = df['sentence2_p'].apply(lambda x: x.split(" "))
df['sentence2_p'] = df['sentence2_p'].apply(lambda x: remove_stops(x))
df['sentence2_p'] = df['sentence2_p'].apply(lambda x: [simplemma.lemmatize(t, langdata) for t in x])

In [96]:
df['sentence1_p_1'] = df['sentence1_p'].apply(lambda x: " ".join(x))
df['sentence2_p_1'] = df['sentence2_p'].apply(lambda x: " ".join(x))


from sklearn.feature_extraction.text import TfidfVectorizer

# Define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(
                        max_features=5000,
                        use_idf=True, 
                        ngram_range=(1,3))

# Fit the vectorizer to synopses texts
tfidf_matrix = tfidf_vectorizer.fit_transform(df['sentence1_p_1'].tolist() + df['sentence2_p_1'].tolist()) 

print("TF-IDF matrix shape: {}".format(tfidf_matrix.shape))



TF-IDF matrix shape: (1000, 5000)


In [97]:
terms = tfidf_vectorizer.get_feature_names()

from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

from sklearn.cluster import KMeans

num_clusters = 2
km = KMeans(n_clusters=num_clusters)

# Perform clustering
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()
#print("Clusters: {}".format(clusters))

C:\Users\Nejc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [98]:
for i in range(500):
    if clusters[i] == clusters[i+500]:
        df["class"].iloc[i] = 1
    else:
        df["class"].iloc[i] = 0

C:\Users\Nejc\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [99]:
df_fin = df.drop(["sentence1_p_1", "sentence2_p_1", "Unnamed: 0"], axis=1)
df_fin = df_fin.rename({"sentence1_p": "sentence1_processed", "sentence2_p": "sentence2_processed"}, axis=1)
df_fin.to_csv("proc/{}.csv".format(now))